In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.functional as F
import torch.nn as nn

class BasicCNN(nn.Module):
	def __init__(self):
		super(BasicCNN, self).__init__()
		"""
			input   - (3, 32, 32)
			block 1 - (32, 32, 32)
			maxpool - (32, 16, 16)
			block 2 - (64, 16, 16)
			maxpool - (64, 8, 8)
			block 3 - (128, 8, 8)
			maxpool - (128, 4, 4)
			block 4 - (128, 4, 4)
			avgpool - (128, 1, 1), reshpe to (128,)
			fc      - (128,) -> (10,)
		"""
		# block 1
		self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
		self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
		self.bn1 = nn.BatchNorm2d(32)

		# block 2
		self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
		self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
		self.bn2 = nn.BatchNorm2d(64)

		# block 3
		self.conv5 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
		self.conv6 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
		self.bn3 = nn.BatchNorm2d(128)

		# block 4
		self.conv7 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
		self.conv8 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
		self.bn4 = nn.BatchNorm2d(256)

		self.avgpool = nn.AdaptiveAvgPool2d(1)
		self.fc = nn.Linear(256, 10)

	def forward(self, x):

		# block 1
		x = F.relu(self.conv1(x))
		x = F.relu(self.bn1(self.conv2(x)))

		# maxpool
		x = F.max_pool2d(x, 2)

		# block 2
		x = F.relu(self.conv3(x))
		x = F.relu(self.bn2(self.conv4(x)))

		# maxpool
		x = F.max_pool2d(x, 2)

		# block 3
		x = F.relu(self.conv5(x))
		x = F.relu(self.bn3(self.conv6(x)))

		# maxpool
		x = F.max_pool2d(x, 2)

		# block 4
		x = F.relu(self.conv7(x))
		x = F.relu(self.bn4(self.conv8(x)))

		# avgpool and reshape to 1D
		x = self.avgpool(x)
		x = x.view(x.size(0), -1)

		# fc
		x = self.fc(x)

		return x

class BasicNN(nn.Module):
	def __init__(self):
		super(BasicNN, self).__init__()

		self.fc1 = nn.Linear(28*28, 512)
		self.bn1 = nn.BatchNorm1d(512)

		self.fc2 = nn.Linear(512, 512)
		self.bn2 = nn.BatchNorm1d(512)

		self.fc3 = nn.Linear(512, 256)
		self.bn3 = nn.BatchNorm1d(256)


		self.fc4 = nn.Linear(256, 128)
		self.bn4 = nn.BatchNorm1d(128)

		self.fc5 = nn.Linear(128, 64)
		self.bn5 = nn.BatchNorm1d(64)



		self.fc6 = nn.Linear(64, 10)

	def forward(self, x):
		x = F.relu(self.bn1(self.fc1(x)))

		x = F.relu(self.bn2(self.fc2(x)))
		x = F.relu(self.bn3(self.fc3(x)))

		x = F.relu(self.bn4(self.fc4(x)))
		x = F.relu(self.bn5(self.fc5(x)))


		x = self.fc6(x)

		return x
class DenoisingAutoEncoder(nn.Module):
    def __init__(self):
        super(DenoisingAutoEncoder,self).__init__()
        self.encoder1 = nn.Sequential(
                        nn.Conv2d(3,32,(3,3),padding=1),
                        nn.ReLU(),
                        nn.BatchNorm2d(32),
                        nn.ReLU(),
                        nn.MaxPool2d(2,2),
                        nn.ZeroPad2d(8),
                        nn.Conv2d(32,32,(3,3),padding=1), 
                        nn.BatchNorm2d(32),
                        nn.ReLU(),
                        nn.MaxPool2d(2,2),
                        nn.ZeroPad2d(8)
        )
        self.encoder2 = nn.Sequential(
                        nn.Conv2d(32,64,(3,3),padding=1),
                        nn.BatchNorm2d(64),
                        nn.ReLU(),
                        nn.Conv2d(64,128,(3,3),padding=1), 
                        nn.BatchNorm2d(128),
                        nn.ReLU(),
        )
        self.decoder = nn.Sequential(
                        nn.ConvTranspose2d(128,64,3,padding=1),
                        nn.BatchNorm2d(64),
                        nn.ReLU(),
                        nn.ConvTranspose2d(64,32,3,padding=1),
                        nn.BatchNorm2d(32),
                        nn.ReLU(),
                        nn.ConvTranspose2d(32,3,3,padding=1),
                        nn.BatchNorm2d(3),
                        nn.Sigmoid()
        )
        
                
    def forward(self,x):
        out = self.encoder1(x)
        out = self.encoder2(out)
        out = self.decoder(out)
        return out

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out
    
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        #self.denoised_layer = denoise()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        #out = self.denoised_layer(x)
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out
		

class EnhancedResnet(nn.Module):
    def __init__(self):
        super(EnhancedResnet, self).__init__()
        self.denoised_layer = DenoisingAutoEncoder()
        self.residualnet = ResNet(Bottleneck, [3,4,6,3])

    def forward(self, x):
        #out = self.denoised_layer(x)
        out = self.denoised_layer(x)
        out = self.residualnet(out)
        return out	

In [ ]:
import sys
import os
import torch
import numpy as np
import torchvision.datasets as datasets
import torchvision.transforms as transforms
# from models.enhanced_resnet import EnhancedResnet
import matplotlib.pyplot as plt
from torchvision import models
import torch
import cv2
from scipy.optimize import differential_evolution
import torch.nn as nn
from torch.autograd import Variable
# from model import BasicCNN
from torchvision.utils import save_image
# from models.enhanced_resnet import EnhancedResnet
from threading import Thread

In [ ]:
enm = EnhancedResnet()
dnl = torch.load('/content/denoiser.pth')
enm.denoised_layer.load_state_dict(dnl['model'])
enm.denoised_layer.eval()

In [ ]:
def plot_figs(imgs):
    f = plt.figure(figsize=(8,4))
    plt.axis('off')
    tot = len(imgs)
    i=0
    pilTrans = transforms.ToPILImage()
    for img in imgs:
        i = i+1
        f.add_subplot(1,tot, i)
        plt.imshow(pilTrans(img))
    plt.show()

In [ ]:
def preprocess(img):
    img = img.astype(np.float32)
    img /= 255.0
    img = img.transpose(2, 0, 1)
    return img
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()
def scale(x, scale=5):
    return cv2.resize(x, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)

In [ ]:
def perturb(x):
    adv_img = img.copy()
    pixs = np.array(np.split(x, len(x)/5)).astype(int)
    loc = (pixs[:, 0], pixs[:,1])
    val = pixs[:, 2:]
    adv_img[loc] = val
    return adv_img
def optimize(x):
    adv_img = perturb(x)
    inp = Variable(torch.from_numpy(preprocess(adv_img)).float().unsqueeze(0))
    out = model(inp)
    prob = softmax(out.data.numpy()[0])
    return prob[pred_orig]

In [ ]:
tr = datasets.CIFAR10('./data', train=False, download=True, transform=transforms.ToTensor())
pred_adv = 0
prob_adv = 0
def callback(x, convergence):
    global pred_adv, prob_adv
    adv_img = perturb(x)
    inp = Variable(torch.from_numpy(preprocess(adv_img)).float().unsqueeze(0))
    out = model(inp)
    prob = softmax(out.data.numpy()[0])
    pred_adv = np.argmax(prob)
    prob_adv = prob[pred_adv]
    if pred_adv != pred_orig and prob_adv >= 0.9:
        print('Attack successful..')
        print('Prob [%s]: %f' %(cifar10_class_names[pred_adv], prob_adv))
        print()
        return True
    else:
        print('Prob [%s]: %f' %(cifar10_class_names[pred_orig], prob[pred_orig]))

# Select Image from the Dataset to attack

In [ ]:
idx = 423
fname = "/content/adv_img_"+str(idx)+".png"
d = 1
iters = 600
popsize = 10

In [ ]:
tnsr,lb = tr.__getitem__(idx)
save_image(tnsr,"/home/testing.png")
image_path = "/home/testing.png" #images/airplane.png(id-438) or car.png adv-3.png adv-37.png adv-158.png 221.png
cifar10_class_names = {0: 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat', 4: 'deer', 5: 'dog', 6: 'frog', 7: 'horse', 8: 'ship', 9: 'truck'}

In [ ]:
orig = cv2.imread(image_path)[..., ::-1]
orig = cv2.resize(orig, (32, 32))
img = orig.copy()
shape = orig.shape
model = BasicCNN()
# saved = torch.load("/cifar10_basiccnn.pth.tar")
# model.load_state_dict(saved['state_dict'])
model.eval()
inp = Variable(torch.from_numpy(preprocess(img)).float().unsqueeze(0))
prob_orig = softmax(model(inp).data.numpy()[0])
pred_orig = np.argmax(prob_orig)
print('Prediction of the image before attack: %s' %(cifar10_class_names[pred_orig]))
#print('Probability: %f' %(prob_orig[pred_orig]))

In [ ]:
bounds = [(0, shape[0]-1), (0, shape[1]), (0, 255), (0, 255), (0, 255)] * d
result = differential_evolution(optimize, bounds, maxiter=iters, popsize=popsize, tol=1e-5, callback=callback)
adv_img = perturb(result.x)
inp = Variable(torch.from_numpy(preprocess(adv_img)).float().unsqueeze(0))
out = model(inp)
prob = softmax(out.data.numpy()[0])
print('Prob [%s]: %f --> Prob[%s]: %f' %(cifar10_class_names[pred_orig], prob_orig[pred_orig], cifar10_class_names[pred_adv], prob_adv))

In [ ]:
from google.colab.patches import cv2_imshow
cv2.imwrite(fname, adv_img[..., ::-1]) #images/adv_img_airplane.png
cv2_imshow(scale(adv_img[..., ::-1]))
while True:
    key = cv2.waitKey(33)
    if key == 27 or key == 32:
        cv2.destroyAllWindows()
        break

Adding autoencoding layers to the model

In [ ]:
orig = cv2.imread(fname)[..., ::-1] #images/adv_img_airplane.png or cat.png
orig = preprocess(orig)
inp = Variable(torch.from_numpy(orig)).float().unsqueeze(0)
out = enm.denoised_layer(inp)
z = torch.reshape(out,(3,32,32))
print("Denoised Image:")
plot_figs([z])

In [ ]:
inp = Variable(z).float().unsqueeze(0)
out = model(inp)
prob = softmax(out.data.numpy()[0])

In [ ]:
n = cifar10_class_names[np.argmax(prob)]
#v = np.amax(prob)
print('Prediction of the image before attack: %s' %(cifar10_class_names[pred_orig]))
#print('Probability: %f' %(prob_orig[pred_orig]))
#if n==cifar10_class_names[pred_orig]:
print('With denoised layers, image is predicted as %s'%(n))
#else:
#    v = prob[pred_orig]
#    print('Confidence in original class is restored to: %f'%(v))

In [ ]:
n = cifar10_class_names[np.argmax(prob)]
v = np.amax(prob)
print('Prediction before attack: %s' %(cifar10_class_names[pred_orig]))
print('Probability: %f' %(prob_orig[pred_orig]))
if n==cifar10_class_names[pred_orig]:
    print('With denoised layers, image is predicted as %s with confidence %f'%(n,v))
else:
    v = prob[pred_orig]
    print('Confidence in original class is restored to: %f'%(v))

In [ ]:
n

In [ ]:
v